In [21]:
# テキストをリスト単位で取り込み、リスト単位で形態素解析

import MeCab
import fitz
import re

doc = fitz.open("知財・無形資産ガバナンスガイドライン_ver2.pdf") # ドキュメントを開く

#ページ単位でテキストをリストに格納
page_text=[]
for page in doc:
    text = page.get_text() # プレーンテキストを取得
    if text != '' :
        #無駄な改行を削除の上で'。'で改行、一文の範囲を明確にする
        text = re.sub(r'。','。\n', re.sub(r'\n','', text))
        #リストに格納
        page_text.append(text)
print(page_text[1])
print() 
print(page_text[2])

  Executive Summary                                                      1 Ⅰ はじめに 5 (１) 本ガイドラインの位置付け 5 (２) 本ガイドラインの想定利用者と期待される活用方法 12 (３) 本ガイドラインにおける「知財・無形資産の投資・活用」のスコープ 15 Ⅱ 日本における知財・無形資産の投資・活用の現状と課題 16 (１) 企業における戦略構築・ガバナンス等の課題 17 (２) 投資家・金融機関における中長期的視点の欠如 20 (３) ジャパン・パッシングの問題 20 (４) 企業と投資家・金融機関の間の思考構造のギャップ 22 Ⅲ 企業価値を顕在化するコミュニケーション・フレームワーク 25 (１) 知財・無形資産の投資・活用を企業変革につなげる「ストーリー」の構築と磨き上げ 27 (２) 知財・無形資産の投資・活用と事業価値をつなぐ因果パスの企図・実装 29 (３) 経営指標（ROIC 等）と知財・無形資産の投資・活用戦略の紐付け 32 Ⅳ 企業に求められる知財・無形資産の投資・活用戦略の構築・開示・発信 34 (１) 企業が意識すべき投資家や金融機関が重視する視点 35 ①知財・無形資産を「価格決定力」「ゲームチェンジ」につなげる 35 ②知財・無形資産の投資・活用を「費用」でなく「資産」形成として捉える 37 ③「ロジック/ストーリー」としての説得的な説明 37 ④全社横断的な体制整備とガバナンス構築 37 (２) 企業における知財・無形資産の投資・活用にかかる戦略構築の流れ 38 ①自社の現状のビジネスモデルと強みとなる知財・無形資産の把握・分析 38 ②知財・無形資産を活用した持続的成長に繋がるビジネスモデルの検討 39 ③競争優位を支える知財・無形資産の維持・強化に向けた戦略の構築 41 ④スタートアップに対する経営資源提供を通じた価値協創能力の構築 41 ⑤サプライチェーンとのパートナーシップにおける外部の知財・無形資産の有効活用 47 (３) 多様な投資家・金融機関に対する開示・発信・対話の実行 48 ①定性的・定量的な説明（KPI 等含む） 48 ②様々な媒体を通じた戦略の開示・発信 50 ③セグメント単位の開示・発信 51 ④投資家との双方向の対話の実践 

In [11]:
def noise_eliminator(text):
#形態素解析の前に、無駄な記号やヘッダ・フッタ等の文言をテキストから除外

    replaced_text = text

    replaced_text = re.sub(r'Panasonic Holdings   統合報告書2023 \d+', ' ', replaced_text)   # ヘッダ・フッタ等の除去
    replaced_text = re.sub(r'\d+ Panasonic Holdings   統合報告書2023', ' ', replaced_text)   # ヘッダ・フッタ等の除去
    replaced_text = re.sub(r'パナソニックグループについて　￨　トップメッセージ　￨　セグメント別戦略　￨　テーマ別戦略　￨　コーポレート・ガバナンス　￨　企業データ', ' ', replaced_text)   # ヘッダ・フッタ等の除去

    replaced_text = re.sub(r'[【】]', ' ', replaced_text)       # 【】の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[〇●◇◆□■△▲▽▼▫▪▹▶▸]', ' ', replaced_text)   # 〇・□等の除去

    replaced_text = re.sub(r'[\d\-]+年度末', '', replaced_text)  # 年度末の除去
    replaced_text = re.sub(r'[\d\-]+年度', '', replaced_text)  # 年度の除去
    replaced_text = re.sub(r'\d+[年月日]', '', replaced_text)  # 年月日の除去
    replaced_text = re.sub(r'\d+回', '', replaced_text)  # \d回（取締役の任命回数など）の除去

    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'https?://[\w/;%#\$\&\?\(\)~\.=\+\-]+', '', replaced_text)  # リンクの除去
    #replaced_text = re.sub(r'\d+\.*\d*', '', replaced_text) #　数字を除去
    #replaced_text = text.lower() #　すべて小文字に変換

    return replaced_text


In [12]:
#形態素解析関数（入力：テキスト、出力：リスト）

def mecab_tokenizer(text):

    path1 = "-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
    path2 = " -u /work_dir/userdic/user.dic"
    mecab = MeCab.Tagger(path1+path2)
   
    parsed_lines = mecab.parse(text).split("\n")[:-2]
    #[:-2]で文末の'EOS',''の取り込みを抑制
    #print(parsed_lines)
    
    #形態素の取得（リスト形式）
    #原形を取得
    token_list = [l.split("\t")[2] for l in parsed_lines]
    #表層形を確認したい場合
    #token_list = [l.split('\t')[0] for l in parsed_lines]
    #print(token_list)
    
    #品詞区分による絞り込み
    #--------------------------------------------
    #Chasenの品詞区分を個別に指定する場合
    
    #品詞区分の取得（リスト形式）
    pos = [l.split('\t')[3] for l in parsed_lines]

    #抽出する品詞区分の定義（完全一致で指定）
    target_pos = ['名詞-一般',
                  '名詞-固有名詞-一般',
                  '名詞-固有名詞-人名-一般',
                  '名詞-固有名詞-人名-姓',
                  '名詞-固有名詞-人名-名',
                  '名詞-固有名詞-組織',
                  '名詞-固有名詞-地域-一般',
                  '名詞-固有名詞-地域-国',
                  '名詞-代名詞-一般',
                  '名詞-代名詞-縮約',
                  '名詞-副詞可能',
                  '名詞-サ変接続',
                  '名詞-形容動詞語幹',
                  '名詞-ナイ形容詞語幹'
                 ]
    #--------------------------------------------
    #Chasenの品詞区分の大項目のみで絞り込む場合
    #品詞区分の取得（リスト形式）
    #pos = [l.split('\t')[3].split("-")[0]  for l in parsed_lines]
    
    #絞り込む品詞区分の定義
    #target_pos = ['名詞']
    
    #--------------------------------------------
    #形態素と品詞区分のリストをペアにしてタプルリスト化、該当する品詞区分の形態素のみリストに出力
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]
    
    #stopwordsの指定
    with open("Japanese.txt","r") as f:
        stopwords1 = f.read().split("\n")
        #stopwordsを直接反映したい場合は以下のリストに記載
        stopwords2 =["以下","ため","当社","当行","場合","影響","可能性",
            "状況","グループ","こと","平成","令和","年","月","期","当","他",
            "等","お","これ","%","以上","もの","株式会社",
            "もの","とも","ある","よる","的","化","お呼び",
            "CEO","会長","社長","副社長","専務","役員","常務","代表社員"
            "代表取締役会長","代表取締役社長","代表取締役","常務取締役","社外取締役","取締役会長","取締役社長",
            "代表執行役員","専務執行役員","常務執行役員","執行役員",
            "取締役","取締役会","監査役","監査役会","議長","所長",
            "担当","100％","100%","株","データ","男性","チーフ","オフィサー","CFO",
            "入社","入所","就任","選任","指名","任命","解任","諮問","答申",
            "*"]
        stopwords = list(set(stopwords1+stopwords2))
    
    # stopwordsの除去
    token_list = [t for t in token_list if t  not in stopwords]
    
    # ひらがなのみの単語を除く
    kana_re = re.compile("^[ぁ-ゖ]+$")
    token_list = [t for t in token_list if not kana_re.match(t)]
    
    return token_list

In [ ]:
#ページのインデックス付きで、形態素解析結果をページ単位で格納
tokenized_page_text = [(index, mecab_tokenizer(noise_eliminator(item))) for index, item in enumerate(page_text)]

#インデックスのないもの
tokenized_page_text_noindex = [sublist for _, sublist in tokenized_page_text]
#全頁マージしたもの
merged_tokenized_list = [item for _, sublist in tokenized_page_text for item in sublist]

#形態素分解前の全テキスト
merged_page_text = " ".join(page_text)

#結果を確認する場合に出力)
#print(page_text[1])
#print()
#print(tokenized_page_text[1])
#print (tokenized_page_text_noindex)
print (merged_page_text)


In [ ]:
print(merged_page_text)

In [36]:
import collections
from collections import Counter
import pandas as pd
import itertools

#辞書形式で単語をカウント
counter = Counter(merged_tokenized_list)
#for word, count in counter.most_common(500):
#    print('%s : %s' % (word, count))

# 単語、件数をDataFrameに格納
count_df = pd.DataFrame(list(counter.items()), columns=['単語', '件数'])
# DataFrameを件数でソート
count_df = count_df.sort_values(by='件数', ascending=False)
# 結果をCSVファイルに出力
count_df.to_csv('CGC_Guideline_v2_Word_list.csv', encoding="utf_8_sig", index=False)

In [27]:
#import itertools
#共起の範囲を一つの文中に限定するため、元のテキストを”。”単位で分割し、文単位で形態素に分解・格納
sentences = [mecab_tokenizer(sentence) for sentence in merged_page_text.split("。")]
#print(sentences)

In [37]:
#各分の形態素2つの組み合わせを作る
sentences_combs = [list(itertools.combinations(sentence,2)) for sentence in sentences]
#print(sentences_combs[0])

#組み合わせた2つの形態素の並びをソート
words_combs = [[tuple(sorted(words)) for words in sentence] for sentence in sentences_combs]
#print(words_combs[0][:30])

target_combs = []
for words_comb in words_combs:
    target_combs.extend(words_comb)
#print(target_combs[:30])
    

ct = collections.Counter(target_combs)
#print(ct)

#import pandas as pd
df = pd.DataFrame([{"1番目" : i[0][0], "2番目": i[0][1], "count":i[1]} for i in ct.most_common()])
df.head(30)
df.to_csv('CGC_Guideline_v2_co_count_result.csv', encoding="utf_8_sig", index=False)

In [33]:
def kyoki_word_network(df):
    from pyvis.network import Network
    import pandas as pd

    
    got_net = Network(height="1000px", width="95%", bgcolor="#FFFFFF", font_color="black", notebook=True)

    got_net.force_atlas_2based()
    got_data = df[:600]

    sources = got_data['1番目']#count
    targets = got_data['2番目']#first
    weights = got_data['count']#second

    edge_data = zip(sources, targets, weights)

    for e in edge_data:
        src = e[0]
        dst = e[1]
        w = e[2]

        got_net.add_node(src, src, title=src)
        got_net.add_node(dst, dst, title=dst)
        got_net.add_edge(src, dst, value=w)

    neighbor_map = got_net.get_adj_list()

    for node in got_net.nodes:
        node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
        node["value"] = len(neighbor_map[node["id"]])

    got_net.show_buttons(filter_=['physics'])
    return got_net

In [34]:
got_net = kyoki_word_network(df)
got_net.show("kyoki.html")

In [35]:
# サンプルのリスト
my_list = ["apple", "banana", "cherry"]

# 各要素とそのインデックスを組み合わせて新しいリストに格納する
processed_list = [(index, item) for index, item in enumerate(my_list)]

# 処理結果のリストを出力する
print(processed_list)

[(0, 'apple'), (1, 'banana'), (2, 'cherry')]
